# Laboratorium: Systemy analizy sieciowej i wykrywania zagrożeń (NIDS/NDR)
Ten notebook poprowadzi Cię krok po kroku przez stworzenie prototypowego silnika detekcji systemu analizy sieciowej w konwencji Proof of Concept (PoC).

Dla przypomnienia, tabela:


| **ID**   | **Kategoria**        | **Opis wymagania**                                                                                      | **Typ**        | **Proponowany sposób udowodnienia/ dodatkowe komentarze**                                                                                             |
|----------|----------------------|---------------------------------------------------------------------------------------------------------|----------------|-----------------------------------------------------------------------------------------------------------|
| **A.1**  | Analiza flow         | Wczytywanie plików PCAP przy użyciu NFStream.                                                            | Must-have      | -                                                   |
| **A.2**  | Analiza flow         | Dla wczytanych przepływów wyświetlanie podsumowania statystyk flow, takich jak podsumowanie ilości przesłanych pakietów pomiędzy danymi hostami. | Must-have      | -                                                |
| **D.1**  | Detection as a Code  | Implementacja przykładowej reguły detekcyjnej w Pythonie             | Must-have      | Napisanie przykładowej reguły i symulacja przy wykorzystaniu scapy.        |
| **ML.1** | Machine Learning     | Klasyfikacja flow na podstawie cech, takich jak czas trwania, liczba pakietów, protokół (np. z użyciem `scikit-learn`). | Must-have      | Raport generowany przez narzędzie zawiera output z modelu, np. w postaci pewności zwróconej przez model lub wizualizacji działania modelu. |
| **ML.2** | Machine Learning     | Redukcja liczby fałszywych pozytywów (FPR) za pomocą oceny jakości modelu i tuningu hiperparametrów.     | Must-have      | Liczenie metryk takich jak FPR, TPR lub wizualizacja macierzy konfuzji dla testowanego przypadku. |
| **E.1**  | Enrichment           | Pobieranie podstawowych informacji o IP/domenach, np. z `geopy` lub innych źródeł Threat Intelligence przy użyciu API. | Nice-to-have      | Enrichment widoczny w raporcie generowanym przez narzędzie.                                               |
| **V.1**  | Wizualizacja         | Mapa geograficzna przedstawiająca lokalizację adresów IP wykrytych jako podejrzane.                     | Nice-to-have   | Wizualizacja lokalizacji IP na mapie, np. przy użyciu bibliotek `folium` lub `plotly`.                     |


##A.1. Wczytywanie plików PCAP przy użyciu NFStream
Będziemy realizować poszczególne wymagania z tabeli zamieszczonej w instrukcji, zaczynając od A.1: Wczytywanie plików PCAP przy użyciu NFStream.

**Przygotowanie środowiska** - najpierw instalujemy potrzebne biblioteki, które możemy przewidzieć na start

In [ ]:
# Instalacja wymaganych pakietów
!pip install nfstream pandas matplotlib scapy

**Krok 1:** Pobranie przykładowych plików PCAP

NFStream to potężna biblioteka, która umożliwia przetwarzanie danych sieciowych na poziomie przepływów (flows). Zaczniemy od pobrania dwóch przykładowych plików PCAP do analizy - jeden zawierający normalny ruch sieciowy, drugi zawierający złośliwy ruch.

In [ ]:
# Pobranie plików z ruchem normalnym
!wget -O normal1.pcap https://mcfp.felk.cvut.cz/publicDatasets/CTU-Normal-13/2017-07-03_capture-win2.pcap
!wget -O normal2.pcap https://mcfp.felk.cvut.cz/publicDatasets/CTU-Normal-14/2017-07-23_capture-winFull.pcap
!wget -O normal3.pcap https://mcfp.felk.cvut.cz/publicDatasets/CTU-Normal-24/2017-04-19_win-normal.pcap

# Pobranie plików z ruchem złośliwym
!wget -O malicious1.pcap https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-14/2013-10-18_capture-win15.pcap
!wget -O malicious2.pcap https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-9/2013-08-20_captureWin5.pcap
!wget -O malicious3.pcap https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-48/botnet-capture-20110816-sogou.pcap
!wget -O malicious4.pcap https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-2/2013-08-20_capture-win2.pcap

# Instalacja mergecap (jeśli jeszcze nie jest zainstalowany)
!apt-get update && apt-get install -y wireshark-common

# Połączenie plików PCAP w jeden plik normalny i jeden złośliwy
!mergecap -w normal_traffic.pcap normal1.pcap normal2.pcap normal3.pcap
!mergecap -w malicious_traffic.pcap malicious1.pcap malicious2.pcap malicious3.pcap malicious4.pcap


**Krok 2**: Podstawowe wczytanie pliku PCAP przy użyciu NFStream

Najpierw wczytamy (z systemu do środowiska Pythona) plik PCAP z normalnym ruchem i przeanalizujemy podstawowe informacje o przepływach.

In [ ]:
# Import bibliotek
from nfstream import NFStreamer
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

# Wczytanie pliku PCAP z normalnym ruchem przy użyciu podstawowych ustawień NFStream
print("Wczytywanie pliku PCAP z normalnym ruchem (domyślne ustawienia)...")
normal_streamer = NFStreamer(source="normal_traffic.pcap")
normal_traffic = normal_streamer.to_pandas()

# Wyświetlenie informacji o DataFrame (normalny ruch)
print(f"\n✅ Wczytano {len(normal_traffic)} przepływów z normal_traffic.pcap")
print("\nKolumny dostępne w danych (ruch normalny):")
print(sorted(normal_traffic.columns.tolist()))

print("\nPierwszych 5 przepływów (ruch normalny):")
display(normal_traffic.head(8))

# Zapisanie listy kolumn do późniejszego porównania
basic_columns = set(normal_traffic.columns.tolist())
print(f"\nLiczba dostępnych kolumn z domyślnymi ustawieniami (normal): {len(basic_columns)}")

# --- ANALIZA LUSTARZANA DLA RUCHU ZŁOŚLIWEGO ---

# Wczytanie pliku PCAP ze złośliwym ruchem
print("\nWczytywanie pliku PCAP ze złośliwym ruchem (domyślne ustawienia)...")
malicious_streamer = NFStreamer(source="malicious_traffic.pcap")
malicious_traffic = malicious_streamer.to_pandas()

# Wyświetlenie informacji o DataFrame (ruch złośliwy)
print(f"\n✅ Wczytano {len(malicious_traffic)} przepływów z malicious_traffic.pcap")
print("\nKolumny dostępne w danych (ruch złośliwy):")
print(sorted(malicious_traffic.columns.tolist()))

print("\nPierwszych 5 przepływów (ruch złośliwy):")
display(malicious_traffic.head(8))

# Zapisanie listy kolumn i porównanie z ruchem normalnym
malicious_columns = set(malicious_traffic.columns.tolist())
print(f"\nLiczba dostępnych kolumn z domyślnymi ustawieniami (malicious): {len(malicious_columns)}")


Znaczenie kolumn (atrybutów) utworzonych przez NFStream jest zgodne z listą pod tym linkiem: https://www.nfstream.org/docs/api#nflow-core-features

**Zadanie do wykonania 1:** Włączenie analizy statystycznej

**Problem:** W podstawowej konfiguracji NFStream nie włącza domyślnie zaawansowanej analizy statystycznej przepływów. Brakuje informacji takich jak:

Entropia (losowość) bajtów w przepływie
Statystyki rozmiaru pakietów (min, max, średnia, odchylenie standardowe)
Rozkład długości pakietów
i wiele innych przydatnych metryk do analizy bezpieczeństwa.

**Zadanie:**

1. Przejrzyj dokumentację NFStream (https://nfstream.org/) i znajdź parametr, który włącza analizę statystyczną przepływów.
2. Zmodyfikuj poniższy kod, aby wczytać ten sam plik PCAP, ale z włączoną analizą statystyczną.
3. Porównaj kolumny dostępne w nowym DataFrame z poprzednim (basic_columns).
4. Zidentyfikuj i wyświetl 5 nowych kolumn statystycznych, które pojawiły się w danych.
5. Zastanów się, które z tych nowych metryk mogą być przydatne do wykrywania anomalii w sieci.

In [ ]:
# Zakładamy, że wcześniej istnieje już DataFrame bez analizy statystycznej i jego kolumny są zapisane w basic_columns

from nfstream import NFStreamer
import pandas as pd

# Wczytanie pliku PCAP z włączoną analizą statystyczną
print("Wczytywanie pliku PCAP z włączoną analizą statystyczną...")

stats_streamer = NFStreamer(
    source="normal_traffic.pcap",
    # TODO: Włącz analizę statystyczną dodając odpowiedni parametr na podstawie dokumentacji NFStream
    # Podpowiedź: Szukaj w dokumentacji argumentów klasy NFStreamer związanych ze statystykami
)

# Konwersja danych do pandas DataFrame
stats_traffic = stats_streamer.to_pandas()
print(f"Wczytano {len(stats_traffic)} przepływów.")

# Porównanie kolumn z poprzednią wersją bez statystyk
stats_columns = set(stats_traffic.columns.tolist())
# TODO: Oblicz zestaw nowych kolumn, które pojawiły się po włączeniu analizy statystycznej
# Wskazówka: użyj operatora różnicy zbiorów
new_columns = ...

# Wyświetl nowe kolumny
print(f"\nLiczba nowych kolumn po włączeniu analizy statystycznej: {len(new_columns)}")
print("\nNowe kolumny statystyczne:")
# TODO: Posortuj i wyświetl listę nowych kolumn
...

# TODO: Wybierz 4-5 ciekawych metryk statystycznych z nowo dodanych kolumn
# Podpowiedź: np. minimalny, maksymalny rozmiar pakietu, odchylenie standardowe itp.
interesting_stats = [
    # np. 'bidirectional_min_ps', ...
]

# Wyświetlenie tych kolumn dla pierwszych 8 wierszy
print("\nWyniki dla wybranych kolumn statystycznych:")
display(stats_traffic[interesting_stats].head(8))


**Zadanie do wykonania 2:** Filtrowanie ruchu sieciowego
NFStream oferuje również możliwość filtrowania pakietów przy użyciu filtrów BPF (Berkeley Packet Filter), co jest przydatne, gdy chcemy skupić się tylko na określonym typie ruchu.
Zadanie:

1. Zmodyfikuj kod, aby:
  - Wczytać tylko ruch TCP na portach 80 i 443 (HTTP i HTTPS)
  - Włączyć analizę statystyczną


1. Porównaj, ile przepływów zostało wczytanych po zastosowaniu filtra w stosunku do liczby wszystkich przepływów
2. Wyświetl statystyki procentowe - jaki procent całego ruchu stanowi ruch HTTP/HTTPS?

In [ ]:
# Zadanie: Sprawdź, jaki procent całego ruchu stanowi ruch HTTP/HTTPS

from nfstream import NFStreamer
import pandas as pd

# Wczytanie wszystkich przepływów z analizą statystyczną
full_streamer = NFStreamer(
    source="normal_traffic.pcap",
    statistical_analysis=True
)
all_flows = full_streamer.to_pandas()
all_flows_count = len(all_flows)
print(f"Całkowita liczba przepływów: {all_flows_count}")

# Wczytanie tylko wybranego ruchu (HTTP i HTTPS)
# TODO: Wczytaj plik jeszcze raz, ale z filtrem BPF ograniczającym ruch do portów 80 i 443
# Podpowiedź: Sprawdź w dokumentacji NFStream, jak przekazać filtr BPF do klasy NFStreamer
http_streamer = NFStreamer(
    ... # Plik źródłowy
    ... # Analiza statystyczna
    # Tutaj wpisz parametr odpowiedzialny za filtr BPF i jego wartość
    ...
)

http_flows = http_streamer.to_pandas()
http_flows_count = len(http_flows)
print(f"Liczba przepływów HTTP/HTTPS: {http_flows_count}")

# Obliczenie procentu
# TODO: Oblicz, jaki procent całkowitego ruchu stanowi wyfiltrowany ruch HTTP/HTTPS
# Wskazówka: użyj dzielenia i pomnóż przez 100
http_percentage = ...
print(f"Ruch HTTP/HTTPS stanowi {http_percentage:.2f}% całkowitego ruchu sieciowego")


**Zadanie do wykonania 3:** Porównanie normalnego i złośliwego ruchu
Mając dwa różne pliki PCAP - z normalnym i złośliwym ruchem - możemy przeprowadzić wstępne porównanie ich charakterystyk.

**Zadanie:**

1. Wczytaj plik z złośliwym ruchem (malicious_traffic.pcap) z włączoną analizą statystyczną
2. Porównaj podstawowe statystyki obu plików (liczba przepływów, średnia liczba pakietów, średni rozmiar pakietów)
3. Wybierz 3 metryki statystyczne i porównaj ich rozkłady między normalnym a złośliwym ruchem
4. Zastanów się, które metryki mogą być najbardziej przydatne do rozróżnienia normalnego ruchu od złośliwego

In [ ]:
# Wczytanie normalnego ruchu z analizą statystyczną (jeśli nie zostało wcześniej wykonane)
if 'stats_traffic' not in locals():
    normal_stats_streamer = NFStreamer(
        source="normal_traffic.pcap",
        statistical_analysis=True
    )
    stats_traffic = normal_stats_streamer.to_pandas()

# Wczytanie złośliwego ruchu
# TODO: Wczytaj plik malicious_traffic.pcap z włączoną analizą statystyczną
# Podpowiedź: użyj NFStreamer, tak jak wyżej, ale ze zmienionym źródłem
malicious_streamer = ...

malicious_traffic = malicious_streamer.to_pandas()

# Porównanie liczby przepływów
print("=== Porównanie liczby przepływów ===")
print("Normalny ruch:", len(stats_traffic))
print("Złośliwy ruch:", len(malicious_traffic))

# TODO: Oblicz średnią liczbę pakietów na przepływ w obu przypadkach
# Wskazówka: sprawdź kolumnę, która odpowiada za liczbę pakietów w przepływie
# Pytanie pomocnicze: Jakie mogą być różnice między normalnym a złośliwym ruchem?

print("\n=== Średnia liczba pakietów na przepływ ===")
normal_avg_packets = ...  # <- oblicz średnią
malicious_avg_packets = ...
print("Normalny ruch:", normal_avg_packets)
print("Złośliwy ruch:", malicious_avg_packets)

# TODO: Oblicz średni rozmiar pakietu (w bajtach) w obu przypadkach
# Wskazówka: Szukaj metryki opisującej średni rozmiar pakietów (np. "mean")
print("\n=== Średni rozmiar pakietu ===")
normal_avg_ps = ...
malicious_avg_ps = ...
print("Normalny ruch:", normal_avg_ps)
print("Złośliwy ruch:", malicious_avg_ps)

# TODO: Wybierz 3 metryki statystyczne i porównaj ich rozkład dla obu typów ruchu
# Podpowiedź: wybierz metryki, które mogą ujawniać różnice w zachowaniu (np. rozmiary pakietów, entropia)

selected_columns = [
    ...,  # <- wpisz nazwy kolumn, które chcesz porównać
    ...,
    ...
]


## A.2: Wyświetlanie podsumowania statystyk flow
W tej części zajmiemy się analizą i wizualizacją statystyk przepływów sieciowych, w tym podsumowaniem ilości przesłanych pakietów między hostami. Ta funkcjonalność jest kluczowym elementem każdego systemu analizy sieciowej, ponieważ pozwala na szybką identyfikację wzorców komunikacji oraz potencjalnych anomalii.

### Krok 1: Przygotowanie danych
Najpierw załadujmy dane z obydwu plików PCAP i włączmy analizę statystyczną, aby mieć pełny dostęp do wszystkich metryk.

In [ ]:
# Zakładamy, że mamy już zainstalowane potrzebne biblioteki i wczytane pliki PCAP
from nfstream import NFStreamer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Jeśli jeszcze nie wczytano danych, zróbmy to teraz
if 'normal_traffic' not in locals() or 'malicious_traffic' not in locals():
    print("Wczytywanie plików PCAP z analizą statystyczną...")

    # Wczytanie normalnego ruchu
    normal_streamer = NFStreamer(
        source="normal_traffic.pcap",
        statistical_analysis=True
    )
    normal_traffic = normal_streamer.to_pandas()

    # Wczytanie złośliwego ruchu
    malicious_streamer = NFStreamer(
        source="malicious_traffic.pcap",
        statistical_analysis=True
    )
    malicious_traffic = malicious_streamer.to_pandas()

    print(f"Wczytano {len(normal_traffic)} przepływów normalnego ruchu i {len(malicious_traffic)} przepływów złośliwego ruchu.")

### Krok 2: Podstawowe podsumowanie statystyk przepływów
Teraz stworzymy funkcję, która będzie generować podsumowanie statystyk flow dla dowolnego zestawu danych:

In [ ]:
def generate_flow_summary(flows_df, title="Podsumowanie statystyk flow"):
    """
    Generuje podstawowe podsumowanie statystyk przepływów sieciowych.

    Args:
        flows_df: DataFrame zawierający dane o przepływach
        title: Tytuł podsumowania

    Returns:
        dict: Słownik ze statystykami
    """
    if flows_df.empty:
        print("Brak danych do analizy.")
        return {}

    stats = {}

    # Nagłówek podsumowania
    print(f"\n{'='*50}")
    print(f"{title.upper()}")
    print(f"{'='*50}")

    # 1. Podstawowe statystyki
    stats['total_flows'] = len(flows_df)
    stats['unique_src_ips'] = flows_df['src_ip'].nunique()
    stats['unique_dst_ips'] = flows_df['dst_ip'].nunique()
    stats['total_packets'] = flows_df['bidirectional_packets'].sum()
    stats['total_bytes'] = flows_df['bidirectional_bytes'].sum()

    print("\n1. PODSTAWOWE STATYSTYKI:")
    print(f"   Liczba przepływów: {stats['total_flows']:,}")
    print(f"   Liczba unikalnych adresów IP źródłowych: {stats['unique_src_ips']:,}")
    print(f"   Liczba unikalnych adresów IP docelowych: {stats['unique_dst_ips']:,}")
    print(f"   Łączna liczba pakietów: {stats['total_packets']:,}")
    print(f"   Łączna liczba bajtów: {stats['total_bytes']:,} ({stats['total_bytes']/1024/1024:.2f} MB)")

    # 2. Statystyki protokołów
    protocol_stats = flows_df['protocol'].value_counts()
    stats['protocol_stats'] = protocol_stats

    print("\n2. STATYSTYKI PROTOKOŁÓW:")
    for protocol, count in protocol_stats.items():
        percentage = 100 * count / stats['total_flows']
        print(f"   {protocol}: {count} przepływów ({percentage:.1f}%)")

    # 3. Statystyki aplikacji
    if 'application_name' in flows_df.columns:
        app_stats = flows_df['application_name'].value_counts().head(10)
        stats['app_stats'] = app_stats

        print("\n3. TOP 10 APLIKACJI:")
        for app, count in app_stats.items():
            percentage = 100 * count / stats['total_flows']
            print(f"   {app}: {count} przepływów ({percentage:.1f}%)")

    # 4. Średnie wartości
    stats['avg_packets_per_flow'] = flows_df['bidirectional_packets'].mean()
    stats['avg_bytes_per_flow'] = flows_df['bidirectional_bytes'].mean()
    stats['avg_duration_ms'] = flows_df['bidirectional_duration_ms'].mean()

    print("\n4. ŚREDNIE WARTOŚCI:")
    print(f"   Średnia liczba pakietów na przepływ: {stats['avg_packets_per_flow']:.2f}")
    print(f"   Średnia liczba bajtów na przepływ: {stats['avg_bytes_per_flow']:.2f} ({stats['avg_bytes_per_flow']/1024:.2f} KB)")
    print(f"   Średni czas trwania przepływu: {stats['avg_duration_ms']:.2f} ms ({stats['avg_duration_ms']/1000:.2f} s)")

    return stats

# Wygenerujmy podsumowanie dla normalnego ruchu
normal_stats = generate_flow_summary(normal_traffic, "Podsumowanie statystyk normalnego ruchu")

### Krok 3: Analiza komunikacji między hostami
Jednym z kluczowych aspektów analizy ruchu sieciowego jest zrozumienie wzorców komunikacji między hostami. Stwórzmy funkcję, która analizuje przepływy pod kątem ilości przesłanych pakietów pomiędzy parami hostów:

In [ ]:
def analyze_host_communication(flows_df, top_n=10):
    """
    Analizuje komunikację między hostami na podstawie ilości przesłanych pakietów i bajtów.

    Args:
        flows_df: DataFrame zawierający dane o przepływach
        top_n: Liczba topowych par do wyświetlenia

    Returns:
        dict: Słownik z wynikami analizy
    """
    if flows_df.empty:
        print("Brak danych do analizy.")
        return {}

    results = {}

    print(f"\n{'='*50}")
    print(f"ANALIZA KOMUNIKACJI MIĘDZY HOSTAMI")
    print(f"{'='*50}")

    # 1. Agregacja przepływów według par src_ip -> dst_ip
    host_pairs = flows_df.groupby(['src_ip', 'dst_ip']).agg({
        'bidirectional_packets': 'sum',
        'bidirectional_bytes': 'sum',
        'id': 'count'  # liczba przepływów
    }).reset_index()

    # Zmiana nazw kolumn dla czytelności
    host_pairs.columns = ['src_ip', 'dst_ip', 'packets', 'bytes', 'flows']

    # Sortowanie według liczby pakietów (malejąco)
    host_pairs_by_packets = host_pairs.sort_values('packets', ascending=False).head(top_n)
    results['top_pairs_by_packets'] = host_pairs_by_packets

    # Sortowanie według liczby bajtów (malejąco)
    host_pairs_by_bytes = host_pairs.sort_values('bytes', ascending=False).head(top_n)
    results['top_pairs_by_bytes'] = host_pairs_by_bytes

    # Sortowanie według liczby przepływów (malejąco)
    host_pairs_by_flows = host_pairs.sort_values('flows', ascending=False).head(top_n)
    results['top_pairs_by_flows'] = host_pairs_by_flows

    # Wyświetlenie wyników analizy
    print(f"\n1. TOP {top_n} PAR HOSTÓW WEDŁUG LICZBY PAKIETÓW:")
    for i, (_, row) in enumerate(host_pairs_by_packets.iterrows(), 1):
        print(f"   {i}. {row['src_ip']} -> {row['dst_ip']}: {row['packets']:,} pakietów ({row['bytes']:,} bajtów w {row['flows']} przepływach)")

    print(f"\n2. TOP {top_n} PAR HOSTÓW WEDŁUG LICZBY BAJTÓW:")
    for i, (_, row) in enumerate(host_pairs_by_bytes.iterrows(), 1):
        print(f"   {i}. {row['src_ip']} -> {row['dst_ip']}: {row['bytes']:,} bajtów ({row['packets']:,} pakietów w {row['flows']} przepływach)")

    return results

# Analiza komunikacji między hostami dla normalnego ruchu
normal_comm = analyze_host_communication(normal_traffic)

### Krok 4: Wizualizacja statystyk flow
Wizualizacje są niezwykle pomocne w szybkiej interpretacji danych. Stwórzmy kilka wykresów, które pomogą lepiej zrozumieć przepływy sieciowe:

In [ ]:
def visualize_flow_statistics(flows_df, title="Wizualizacja statystyk przepływów"):
    """
    Tworzy wizualizacje statystyk przepływów.

    Args:
        flows_df: DataFrame zawierający dane o przepływach
        title: Tytuł dla wizualizacji
    """
    if flows_df.empty:
        print("Brak danych do wizualizacji.")
        return

    # Konfiguracja estetyki wykresów
    sns.set(style="whitegrid")
    plt.figure(figsize=(15, 12))
    plt.suptitle(title, fontsize=16)

    # 1. Wykres rozkładu protokołów
    plt.subplot(2, 2, 1)
    protocol_counts = flows_df['protocol'].value_counts()
    sns.barplot(x=protocol_counts.index, y=protocol_counts.values)
    plt.title('Rozkład protokołów')
    plt.xlabel('Protokół')
    plt.ylabel('Liczba przepływów')
    plt.xticks(rotation=45)

    # 2. Wykres rozkładu aplikacji (jeśli dostępne)
    if 'application_name' in flows_df.columns:
        plt.subplot(2, 2, 2)
        app_counts = flows_df['application_name'].value_counts().head(10)
        sns.barplot(x=app_counts.values, y=app_counts.index)
        plt.title('Top 10 aplikacji')
        plt.xlabel('Liczba przepływów')
        plt.ylabel('Aplikacja')

    # 3. Histogram rozmiaru przepływów (w bajtach)
    plt.subplot(2, 2, 3)
    sns.histplot(flows_df['bidirectional_bytes'], bins=30, kde=True)
    plt.title('Rozkład rozmiaru przepływów')
    plt.xlabel('Rozmiar przepływu [bajty]')
    plt.ylabel('Liczba przepływów')
    plt.xscale('log')  # Skala logarytmiczna dla lepszej wizualizacji

    # 4. Histogram liczby pakietów w przepływach
    plt.subplot(2, 2, 4)
    sns.histplot(flows_df['bidirectional_packets'], bins=30, kde=True)
    plt.title('Rozkład liczby pakietów w przepływach')
    plt.xlabel('Liczba pakietów')
    plt.ylabel('Liczba przepływów')
    plt.xscale('log')  # Skala logarytmiczna dla lepszej wizualizacji

    plt.tight_layout()
    plt.subplots_adjust(top=0.92)
    plt.show()

# Wizualizacja statystyk dla normalnego ruchu
visualize_flow_statistics(normal_traffic, "Wizualizacja statystyk normalnego ruchu")

### Zadanie do wykonania 1: Porównanie statystyk normalnego i złośliwego ruchu
Mając dostęp do danych zarówno z normalnego, jak i złośliwego ruchu, możemy przeprowadzić analizę porównawczą, aby wychwycić różnice.

Zadanie:

1. Wygeneruj podsumowanie statystyk dla złośliwego ruchu przy użyciu funkcji generate_flow_summary
2. Przeanalizuj komunikację między hostami w złośliwym ruchu przy użyciu funkcji analyze_host_communication
3. Stwórz wizualizację statystyk złośliwego ruchu przy użyciu funkcji visualize_flow_statistics
4. Porównaj wyniki dla normalnego i złośliwego ruchu i zidentyfikuj kluczowe różnice

In [ ]:
# Zadanie: Porównaj statystyki normalnego i złośliwego ruchu sieciowego

# Zakładamy, że dane z normalnego ruchu i obiekt normal_stats zostały wcześniej wygenerowane,
# np. za pomocą generate_flow_summary(stats_traffic, "Podsumowanie normalnego ruchu")

# TODO: Wygeneruj podsumowanie statystyk dla złośliwego ruchu
# Podpowiedź: użyj funkcji generate_flow_summary z odpowiednimi argumentami
malicious_stats = ...  # <- uzupełnij

# TODO: Przeanalizuj komunikację między hostami w złośliwym ruchu
# Wskazówka: użyj odpowiedniej funkcji (z zaproponowanych powyżej), nie potrzebujesz dodatkowych parametrów
malicious_comm = ...  # <- uzupełnij

# TODO: Zwizualizuj statystyki przepływów w złośliwym ruchu
# Podpowiedź: funkcja przyjmuje dane + tytuł do wykresu
visualize_flow_statistics(...)  # <- uzupełnij

# Porównanie kluczowych parametrów
comparison_metrics = [
    'total_flows', 'unique_src_ips', 'unique_dst_ips',
    'avg_packets_per_flow', 'avg_bytes_per_flow', 'avg_duration_ms'
]

print("\n=== PORÓWNANIE NORMALNEGO I ZŁOŚLIWEGO RUCHU ===")
for metric in comparison_metrics:
    normal_value = normal_stats.get(metric, 0)
    malicious_value = malicious_stats.get(metric, 0)
    print(f"{metric}: Normal={normal_value:.2f}, Malicious={malicious_value:.2f}, Ratio={malicious_value/normal_value if normal_value else 0:.2f}")


### Zadanie do wykonania 2: Analiza przepływów według kategorii aplikacji
W bardziej zaawansowanej analizie ruchu sieciowego, możemy badać przepływy pogrupowane według kategorii aplikacji, aby zrozumieć, jakie typy ruchu dominują w sieci.

Zadanie:

1. Stwórz funkcję, która grupuje przepływy według kategorii aplikacji (application_category_name) i generuje podsumowanie dla każdej kategorii
2. Zaimplementuj wizualizację, która porównuje kategorie aplikacji pod względem różnych metryk (liczba przepływów, liczba pakietów, liczba bajtów)
3. Zastosuj tę funkcję do danych normalnego i złośliwego ruchu
4. Sformułuj wnioski dotyczące różnic w rozkładzie kategorii aplikacji między normalnym a złośliwym ruchem

In [ ]:
# Zadanie: Porównaj ruch normalny i złośliwy według kategorii aplikacji

import matplotlib.pyplot as plt
import seaborn as sns

def analyze_application_categories(flows_df, title="Analiza kategorii aplikacji"):
    if flows_df.empty or 'application_category_name' not in flows_df.columns:
        print("Brak danych do analizy kategorii aplikacji.")
        return {}

    # Grupowanie przepływów według kategorii aplikacji
    # TODO: Pogrupuj dane i policz: liczbę przepływów, sumę pakietów i bajtów
    # Podpowiedź: użyj groupby() i agg()
    category_stats = flows_df.groupby('application_category_name').agg({
        'id': ...,  # <- policz liczbę przepływów w kategorii
        'bidirectional_packets': ...,  # <- zsumuj liczbę pakietów
        'bidirectional_bytes': ...  # <- zsumuj liczbę bajtów
    }).reset_index()

    # TODO: Zmień nazwy kolumn na bardziej czytelne: category, flows, packets, bytes
    # Wskazówka: użyj category_stats.columns = [...]
    ...

    # TODO: Posortuj dane malejąco według liczby przepływów (użyj metody sort_values, jeśli jest taka potrzeba poszukaj dokumentacji w Google)
    category_stats = ...

    # Wyświetlenie wyników tekstowych
    print(f"\n{'='*50}")
    print(f"{title.upper()}")
    print(f"{'='*50}")

    for _, row in category_stats.iterrows():
        print(f"Kategoria: {row['category']}")
        print(f"   Przepływy: {row['flows']} ({100*row['flows']/flows_df.shape[0]:.1f}%)")
        print(f"   Pakiety: {row['packets']:,} ({100*row['packets']/flows_df['bidirectional_packets'].sum():.1f}%)")
        print(f"   Bajty: {row['bytes']:,} ({100*row['bytes']/flows_df['bidirectional_bytes'].sum():.1f}%)")
        print()

    # Wizualizacja
    plt.figure(figsize=(15, 10))

    # Wykres liczby przepływów według kategorii
    plt.subplot(3, 1, 1)
    sns.barplot(x='flows', y='category', data=category_stats)
    plt.title('Liczba przepływów według kategorii aplikacji')
    plt.xlabel('Liczba przepływów')

    # Wykres liczby pakietów według kategorii
    plt.subplot(3, 1, 2)
    sns.barplot(x='packets', y='category', data=category_stats)
    plt.title('Liczba pakietów według kategorii aplikacji')
    plt.xlabel('Liczba pakietów')

    # Wykres liczby bajtów według kategorii
    plt.subplot(3, 1, 3)
    sns.barplot(x='bytes', y='category', data=category_stats)
    plt.title('Liczba bajtów według kategorii aplikacji')
    plt.xlabel('Liczba bajtów')

    plt.tight_layout()
    plt.suptitle(title, fontsize=16)
    plt.subplots_adjust(top=0.92)
    plt.show()

    return category_stats


# TODO: Uruchom funkcję dla danych z normalnego i złośliwego ruchu
# Podpowiedź: Wystarczy podać DataFrame i tytuł
normal_categories = analyze_application_categories(..., "Ruch normalny")  # <- uzupełnij
malicious_categories = analyze_application_categories(..., "Ruch złośliwy")  # <- uzupełnij

# (Opcjonalnie) Na podstawie wykresów porównaj, które aplikacje dominują w każdym przypadku.


## D.1: Implementacja przykładowej reguły detekcyjnej w Pythonie
W tej części zajmiemy się implementacją prostych reguł detekcyjnych zgodnie z paradygmatem Detection as a Code. Oznacza to, że zamiast definiować statyczne reguły w języku opisu (jak np. Zeek, Snort czy Yara), będziemy używać Pythona do wykrywania potencjalnie złośliwej aktywności w ruchu sieciowym.

### Krok 1: Prosta implementacja reguły detekcyjnej
Zaczniemy od zdefiniowania i wdrożenia prostej reguły detekcyjnej, która będzie analizować każdy przepływ sieciowy:

In [ ]:
def detect_suspicious_flow(flow):
    """
    Wykrywa podejrzane przepływy na podstawie prostych reguł.

    Args:
        flow: Wiersz DataFrame reprezentujący pojedynczy przepływ

    Returns:
        bool: True jeśli przepływ jest podejrzany, False w przeciwnym razie
        str: Powód uznania przepływu za podejrzany (lub None)
    """
    # Domyślnie zakładamy, że przepływ nie jest podejrzany
    is_suspicious = False
    reason = None

    # Reguła 1: Duża liczba pakietów w krótkim czasie (potencjalny DoS)
    if flow['bidirectional_packets'] > 100 and flow['bidirectional_duration_ms'] < 1000:
        is_suspicious = True
        reason = f"Duża liczba pakietów ({flow['bidirectional_packets']}) w krótkim czasie ({flow['bidirectional_duration_ms']} ms)"

    # Reguła 2: Duża ilość danych wysłanych do nietypowego portu
    elif flow['dst_port'] not in [80, 443, 22, 53] and flow['src2dst_bytes'] > 10:
        is_suspicious = True
        reason = f"Duża ilość danych ({flow['src2dst_bytes']} bajtów) wysłana do nietypowego portu {flow['dst_port']}"

    # Reguła 3: Asymetryczny przepływ (dużo więcej danych wysłanych niż odebranych)
    elif flow['src2dst_bytes'] > 0 or flow['dst2src_bytes'] > 0 or flow['src2dst_bytes'] / flow['dst2src_bytes'] > 10:
        is_suspicious = True
        reason = f"Asymetryczny przepływ (stosunek wysłanych/odebranych danych: {flow['src2dst_bytes'] / flow['dst2src_bytes']:.2f})"

    return is_suspicious, reason

### Krok 2: Zastosowanie reguły detekcyjnej do danych
Teraz zastosujmy naszą regułę do analizy wcześniej wczytanych danych:

In [ ]:
#TODO: poprawić reguły, albo wgrać bardziej zaawansowany plik, żeby coś wykrywało

In [ ]:
def detect_suspicious_flow(flow):
    is_suspicious = False
    reason = ""

    # Przykładowe reguły detekcyjne
    if flow['bidirectional_packets'] > 1000:
        is_suspicious = True
        reason = "Duża liczba pakietów"

    elif flow['bidirectional_bytes'] > 1_000_000:
        is_suspicious = True
        reason = "Duży wolumen danych"

    elif flow['src2dst_bytes'] > 0 and flow['dst2src_bytes'] == 0:
        is_suspicious = True
        reason = "Asymetryczny przepływ (brak danych w jednym kierunku)"

    elif flow['dst2src_bytes'] > 0:
        ratio = flow['src2dst_bytes'] / flow['dst2src_bytes']
        if ratio > 100:
            is_suspicious = True
            reason = f"Asymetryczny przepływ (stosunek wysłanych/odebranych danych: {ratio:.2f})"

    return is_suspicious, reason


# Analiza złośliwego ruchu
print("\nAnaliza złośliwego ruchu...")
malicious_suspicious = apply_detection_rule(malicious_traffic, detect_suspicious_flow)

print(f"Wykryto {len(malicious_suspicious)} podejrzanych przepływów w złośliwym ruchu.")
if not malicious_suspicious.empty:
    display(malicious_suspicious.head(10))  # Pokazujemy tylko 10 pierwszych, jeśli jest ich dużo



### Zadanie do wykonania 1: Implementacja prostej reguły detekcyjnej dla skanowania portów

Zadanie:

Zaimplementuj prostą regułę detekcyjną, która będzie wykrywać potencjalne skanowanie portów. Skanowanie portów zazwyczaj charakteryzuje się:

- Małą liczbą pakietów na przepływ (1-3 pakiety)
- Krótkim czasem trwania przepływu (poniżej 500 ms)
- Małą ilością przesłanych danych (poniżej 300 bajtów)

In [ ]:
def detect_port_scan(flow):
    """
    Sprawdza, czy przepływ wygląda na skanowanie portu.
    Zwraca True i opis, jeśli spełnia warunki.
    """
    is_port_scan = False
    reason = None

    # TODO: Sprawdź, czy przepływ spełnia WSZYSTKIE trzy warunki, w razie potrzeby sprawdź nazwy odpowiednich atrybutów w dokumentacji NFStream:
    # - liczba pakietów ≤ 3
    # - czas trwania < 500
    # - liczba bajtów < 300

    if ...:
        if ...:
            if ...:
                is_port_scan = True
                reason = f"Podejrzane skanowanie: {flow['...']} pakiety, " \
                         f"{flow['...']} ms, " \
                         f"{flow['...']} bajtów"

    return is_port_scan, reason


# TODO: Zastosuj regułę do całego DataFrame ze złośliwym ruchem
# Podpowiedź: użyj gotowej funkcji apply_detection_rule, np.:
# apply_detection_rule(dane, funkcja_detekcyjna)

port_scan_results = apply_detection_rule(malicious_traffic, detect_port_scan)

# Wyniki
print(f"Wykryto {len(port_scan_results)} potencjalnych skanowań portów.")

# TODO: Wyświetl kilka pierwszych wykrytych przepływów
if not port_scan_results.empty:
    display(port_scan_results.head(10))


### Zadanie do wykonania 2: Wizualizacja wyników detekcji
Zadanie:
Stwórz prostą wizualizację wyników detekcji, np. wykres przedstawiający liczbę wykrytych podejrzanych przepływów według ich rodzaju (powodu detekcji).

In [ ]:
import matplotlib.pyplot as plt

def visualize_detection_results(suspicious_flows_df):
    """
    Tworzy wykres liczby wykrytych podejrzanych przepływów według ich powodu.

    Args:
        suspicious_flows_df: DataFrame z wykrytymi przepływami (musi zawierać kolumnę 'reason')
    """
    if suspicious_flows_df.empty:
        print("Brak danych do wizualizacji.")
        return

    # TODO: Zlicz, ile razy wystąpił każdy powód detekcji
    # Podpowiedź: użyj value_counts() na kolumnie 'reason'
    reason_counts = ...  # <- uzupełnij

    # TODO: Utwórz wykres słupkowy z tych danych
    # Pytanie pomocnicze: Jaka forma wykresu najlepiej pokazuje porównanie liczebności kategorii?
    plt.figure(figsize=(10, 6))
    ...  # <- narysuj wykres na podstawie reason_counts (użyj metody plot)
    # Ustawienia wykresu (nie musisz zmieniać)
    plt.title('Liczba wykrytych podejrzanych przepływów według powodu')
    plt.xlabel('Powód')
    plt.ylabel('Liczba przepływów')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


# TODO: Wywołaj funkcję tylko jeśli dane z detekcji są dostępne
# Podpowiedź: sprawdź, czy zmienna istnieje i czy DataFrame nie jest pusty
if 'malicious_suspicious' in locals() and not malicious_suspicious.empty:
    visualize_detection_results(...)  # <- podaj odpowiedni argument
else:
    print("Nie znaleziono danych z detekcji. Upewnij się, że wykonałeś analizę wcześniej.")


## ML.1: Klasyfikacja flow na podstawie cech przy użyciu uczenia maszynowego
W tej części zaimplementujemy prosty model uczenia maszynowego do klasyfikacji przepływów sieciowych jako normalne lub złośliwe. Wykorzystamy algorytm Random Forest do tej klasyfikacji i ocenimy jego skuteczność za pomocą macierzy błędów.

### Krok 1: Przygotowanie danych do uczenia maszynowego
Najpierw musimy przygotować dane do treningu modelu, łącząc dane z normalnego i złośliwego ruchu oraz wybierając odpowiednie cechy.

In [ ]:
# Import potrzebnych bibliotek
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Sprawdzenie, czy mamy wczytane dane
if 'normal_traffic' not in locals() or 'malicious_traffic' not in locals():
    print("Nie znaleziono wczytanych danych. Upewnij się, że wykonałeś sekcję A.1.")
else:
    # Krok 1: Dodanie etykiet do danych
    normal_traffic['label'] = 0  # 0 oznacza normalny ruch
    malicious_traffic['label'] = 1  # 1 oznacza złośliwy ruch

    # Krok 2: Połączenie obu zbiorów danych
    combined_traffic = pd.concat([normal_traffic, malicious_traffic], ignore_index=True)

    # Krok 3: Wybór cech do treningu modelu
    selected_features = [
        'bidirectional_packets', 'bidirectional_bytes', 'bidirectional_duration_ms',
        'src2dst_packets', 'src2dst_bytes', 'dst2src_packets', 'dst2src_bytes',
        'protocol'
    ]

    # Sprawdzenie, czy wszystkie wybrane cechy są dostępne w danych
    available_features = [feature for feature in selected_features if feature in combined_traffic.columns]

    if len(available_features) < 3:
        print("Za mało dostępnych cech do treningu modelu. Sprawdź dane wejściowe.")
    else:
        print(f"Dostępne cechy do treningu: {available_features}")

        # Krok 4: Przygotowanie danych do treningu
        X = combined_traffic[available_features]
        y = combined_traffic['label']

        # Obsługa wartości null - wypełniamy je zerami
        X = X.fillna(0)

        print(f"Zbiór danych do treningu: {X.shape[0]} przykładów, {X.shape[1]} cech")
        print(f"Rozkład klas: {y.value_counts().to_dict()}")

### Krok 2: Trenowanie modelu Decision Tree
Teraz wytrenujemy model Decision Tree do klasyfikacji przepływów:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Kontynuacja z poprzedniego kodu (zakładamy, że X i y są już zdefiniowane)
if 'X' in locals() and 'y' in locals():
    # Krok 1: Podział na zbiór treningowy i testowy
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    print(f"Zbiór treningowy: {X_train.shape[0]} przykładów")
    print(f"Zbiór testowy: {X_test.shape[0]} przykładów")

    # Krok 2: Trenowanie modelu Decision Tree
    print("Trenowanie modelu Decision Tree...")
    dt_classifier = DecisionTreeClassifier(random_state=42, max_depth=2)
    dt_classifier.fit(X_train, y_train)

    # Krok 3: Ocena modelu na zbiorze testowym
    y_pred = dt_classifier.predict(X_test)
    accuracy = (y_pred == y_test).mean() * 100

    print(f"Dokładność modelu na zbiorze testowym: {accuracy:.2f}%")

    # Krok 4: Wypisanie ważności cech
    feature_importance = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': dt_classifier.feature_importances_
    }).sort_values('Importance', ascending=False)

    print("\nWażność cech w modelu:")
    display(feature_importance)

    # Krok 5: Wizualizacja ważności cech
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=feature_importance)
    plt.title('Ważność cech w modelu Decision Tree')
    plt.tight_layout()
    plt.show()


### Krok 3: Ocena modelu - macierz konfuzji i raport klasyfikacji
Teraz ocenimy skuteczność naszego modelu za pomocą macierzy konfuzji i raportu klasyfikacji:

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Kontynuacja z poprzedniego kodu
if 'y_pred' in locals() and 'y_test' in locals():
    # Krok 1: Obliczenie macierzy konfuzji
    cm = confusion_matrix(y_test, y_pred)

    # Krok 2: Wizualizacja macierzy konfuzji
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Normalny', 'Złośliwy'],
                yticklabels=['Normalny', 'Złośliwy'])
    plt.title('Macierz konfuzji (Decision Tree)')
    plt.xlabel('Przewidziana klasa')
    plt.ylabel('Prawdziwa klasa')
    plt.tight_layout()
    plt.show()

    # Krok 3: Wypisanie raportu klasyfikacji
    print("\nRaport klasyfikacji (Decision Tree):")
    report = classification_report(y_test, y_pred, target_names=['Normalny', 'Złośliwy'])
    print(report)

    # Krok 4: Obliczenie i wyświetlenie wskaźników
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()

        accuracy = (tp + tn) / (tp + tn + fp + fn)
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

        print(f"\nDokładność (Accuracy): {accuracy:.4f}")
        print(f"Precyzja (Precision): {precision:.4f}")
        print(f"Czułość (Recall): {recall:.4f}")
        print(f"F1 Score: {f1_score:.4f}")
    else:
        print("\n⚠️ Niewłaściwy rozmiar macierzy konfuzji — klasy nie są binarne?")


### Krok 4: Klasyfikacja przepływów i wizualizacja wyników
Na koniec użyjemy naszego modelu do klasyfikacji przepływów i zwizualizujemy wyniki:

In [ ]:
# Kontynuacja z poprzedniego kodu
if 'dt_classifier' in locals():
    # Krok 1: Klasyfikacja wszystkich przepływów
    print("\nKlasyfikacja wszystkich przepływów...")
    combined_traffic['predicted_label'] = dt_classifier.predict(X)

    # Krok 2: Obliczenie prawdopodobieństw przynależności do klas
    combined_traffic['malicious_probability'] = dt_classifier.predict_proba(X)[:, 1]

    # Krok 3: Wizualizacja rozkładu prawdopodobieństw dla obu klas
    plt.figure(figsize=(10, 6))

    # Histogram dla normalnego ruchu
    plt.hist(combined_traffic[combined_traffic['label'] == 0]['malicious_probability'],
             bins=30, alpha=0.5, label='Normalny ruch', color='blue')

    # Histogram dla złośliwego ruchu
    plt.hist(combined_traffic[combined_traffic['label'] == 1]['malicious_probability'],
             bins=30, alpha=0.5, label='Złośliwy ruch', color='red')

    plt.title('Rozkład prawdopodobieństw klasyfikacji (Decision Tree)')
    plt.xlabel('Prawdopodobieństwo bycia złośliwym przepływem')
    plt.ylabel('Liczba przepływów')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Krok 4: Wyświetlenie przykładowych wyników klasyfikacji
    print("\nPrzykładowe wyniki klasyfikacji:")
    sample_results = combined_traffic[['src_ip', 'dst_ip', 'protocol',
                                       'bidirectional_packets', 'bidirectional_bytes',
                                       'label', 'predicted_label', 'malicious_probability']]

    print("\nPrzykłady prawidłowo sklasyfikowanych normalnych przepływów:")
    display(sample_results[(sample_results['label'] == 0) &
                           (sample_results['predicted_label'] == 0)].head(5))

    print("\nPrzykłady prawidłowo sklasyfikowanych złośliwych przepływów:")
    display(sample_results[(sample_results['label'] == 1) &
                           (sample_results['predicted_label'] == 1)].head(5))

    print("\nPrzykłady błędnie sklasyfikowanych przepływów (fałszywe pozytywy):")
    display(sample_results[(sample_results['label'] == 0) &
                           (sample_results['predicted_label'] == 1)].head(5))

    print("\nPrzykłady błędnie sklasyfikowanych przepływów (fałszywe negatywy):")
    display(sample_results[(sample_results['label'] == 1) &
                           (sample_results['predicted_label'] == 0)].head(5))


### Zadanie do wykonania 1: Rozszerzenie zestawu cech

Zadanie:
Rozszerz zestaw cech używanych do treningu modelu, dodając dodatkowe metryki dostępne w danych, takie jak:

- liczbę przesłanych pakietów i bajtów zarówno w jednym, jak i w obu kierunkach komunikacji,
- czas trwania przepływu sieciowego,
- protokół warstwy transportowej używany w komunikacji (np. TCP, UDP),
- oraz informacje o liczbie wystąpień konkretnych flag TCP, takich jak rozpoczęcie połączenia, jego zakończenie, reset czy potwierdzenia transmisji.



Następnie porównaj wyniki klasyfikacji przed i po rozszerzeniu zestawu cech.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# TODO: Zdefiniuj rozszerzony zestaw cech
# Podpowiedź: dodaj cechy związane z pakietami, bajtami, czasem trwania, protokołem i flagami TCP
extended_features = [
    ...
]

# TODO: Sprawdź, które z tych cech faktycznie istnieją w danych
available_extended_features = [f for f in extended_features if f in combined_traffic.columns]

# TODO: Przygotuj dane do modelu
# Wskazówka: użyj .fillna(0) na zbiorze cech
X_extended = ...
y = ...

# TODO: Podziel dane na zbiór treningowy i testowy
X_train_ext, X_test_ext, y_train, y_test = train_test_split(...)

# TODO: Zbuduj i wytrenuj model drzewa decyzyjnego
clf = DecisionTreeClassifier(random_state=42)
clf.fit(...)

# TODO: Wygeneruj predykcje i policz accuracy
y_pred = ...
accuracy_ext = ...
print(f"Dokładność modelu z rozszerzonymi cechami: {accuracy_ext:.2f}%")

# TODO: Oblicz i wypisz precision, recall i F1 score
precision_ext = ...
recall_ext = ...
f1_ext = ...
print(f"Precision: {precision_ext:.4f}")
print(f"Recall:    {recall_ext:.4f}")
print(f"F1-score:  {f1_ext:.4f}")

# TODO: Narysuj macierz konfuzji
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Macierz konfuzji - model z rozszerzonymi cechami")
plt.xlabel("Przewidziana klasa")
plt.ylabel("Rzeczywista klasa")
plt.tight_layout()
plt.show()

# TODO: Wyświetl i zwizualizuj najważniejsze cechy
importance_df = pd.DataFrame({
    'Feature': X_train_ext.columns,
    'Importance': clf.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nTop 10 najważniejszych cech:")
display(importance_df.head(10))

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df.head(10))
plt.title("Najważniejsze cechy (Decision Tree)")
plt.tight_layout()
plt.show()


## ML.2: Poprawa wyników poprzez tuning modelu
W tej części zajmiemy się redukcją liczby fałszywych pozytywów i poprawą wyników w naszym systemie detekcji, poprzez ocenę jakości modelu i dostrajanie jego parametrów. Skupimy się na algorytmie drzewa decyzyjnego (Decision Tree), który jest prostszy do interpretacji i wizualizacji niż Random Forest.

### Krok 1: Przygotowanie danych i trenowanie podstawowego drzewa decyzyjnego
Najpierw przygotujmy dane i wytrenujmy podstawowy model drzewa decyzyjnego:

In [ ]:
# Import potrzebnych bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from IPython.display import display

# Sprawdzenie, czy mamy wczytane dane
if 'normal_traffic' not in locals() or 'malicious_traffic' not in locals():
    print("Nie znaleziono wczytanych danych. Upewnij się, że wykonałeś sekcję A.1.")
else:
    # Przygotowanie danych (jeśli jeszcze nie zostały przygotowane)
    if 'combined_traffic' not in locals():
        # Dodanie etykiet do danych
        normal_traffic['label'] = 0  # 0 oznacza normalny ruch
        malicious_traffic['label'] = 1  # 1 oznacza złośliwy ruch

        # Połączenie obu zbiorów danych
        combined_traffic = pd.concat([normal_traffic, malicious_traffic], ignore_index=True)

    # Wybór cech do treningu modelu
    selected_features = [
        'bidirectional_packets', 'bidirectional_bytes', 'bidirectional_duration_ms',
        'src2dst_packets', 'src2dst_bytes', 'dst2src_packets', 'dst2src_bytes',
        'protocol'
    ]

    # Sprawdzenie, czy wszystkie wybrane cechy są dostępne w danych
    available_features = [feature for feature in selected_features if feature in combined_traffic.columns]

    if len(available_features) < 3:
        print("Za mało dostępnych cech do treningu modelu. Sprawdź dane wejściowe.")
    else:
        print(f"Dostępne cechy do treningu: {available_features}")

        # Przygotowanie danych do treningu
        X = combined_traffic[available_features]
        y = combined_traffic['label']

        # Obsługa wartości null - wypełniamy je zerami
        X = X.fillna(0)

        # Podział na zbiór treningowy i testowy
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        print(f"Zbiór treningowy: {X_train.shape[0]} przykładów")
        print(f"Zbiór testowy: {X_test.shape[0]} przykładów")

        # Trenowanie podstawowego drzewa decyzyjnego (bez ograniczeń)
        dt_classifier = DecisionTreeClassifier(random_state=42, max_depth = 2)
        dt_classifier.fit(X_train, y_train)

        # Ocena modelu na zbiorze testowym
        y_pred = dt_classifier.predict(X_test)
        accuracy = (y_pred == y_test).mean() * 100

        print(f"Dokładność podstawowego drzewa decyzyjnego: {accuracy:.2f}%")

        # Obliczenie macierzy konfuzji
        cm = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = cm.ravel()

        # Obliczenie FPR (False Positive Rate)
        fpr = fp / (fp + tn)

        print(f"Liczba fałszywych pozytywów: {fp}")
        print(f"False Positive Rate (FPR): {fpr:.4f}")

        # Wizualizacja macierzy konfuzji
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['Normalny', 'Złośliwy'],
                   yticklabels=['Normalny', 'Złośliwy'])
        plt.title('Macierz konfuzji - Podstawowe drzewo decyzyjne')
        plt.xlabel('Przewidziana klasa')
        plt.ylabel('Prawdziwa klasa')
        plt.tight_layout()
        plt.show()

### Krok 2: Wizualizacja podstawowego drzewa decyzyjnego
Teraz zwizualizujmy wytrenowane drzewo decyzyjne, aby lepiej zrozumieć, jak podejmuje decyzje:

In [ ]:
# Wizualizacja drzewa decyzyjnego
if 'dt_classifier' in locals() and 'available_features' in locals():
    plt.figure(figsize=(20, 10))
    plot_tree(dt_classifier, filled=True, feature_names=available_features,
              class_names=['Normalny', 'Złośliwy'], max_depth=3, fontsize=10)
    plt.title('Podstawowe drzewo decyzyjne (ograniczone do głębokości 3 dla czytelności)')
    plt.tight_layout()
    plt.show()

    # Sprawdzenie głębokości drzewa
    print(f"Głębokość podstawowego drzewa: {dt_classifier.get_depth()}")
    print(f"Liczba liści podstawowego drzewa: {dt_classifier.get_n_leaves()}")
else:
    print("Nie znaleziono drzewa decyzyjnego. Upewnij się, że kod z kroku 1 został poprawnie wykonany.")

### Zadanie do wykonania 1 - Eksperymentowanie z parametrami drzewa decyzyjnego
Zadanie:

Zbadaj wpływ parametru max_depth (maksymalna głębokość drzewa) na skuteczność detekcji i liczbę fałszywych pozytywów. Przetestuj kilka różnych wartości tego parametru (np. 3, 5, 10, None - gdzie None oznacza nieograniczoną głębokość), wytrenuj model dla każdej wartości i porównaj wyniki.

1. Dla każdej wartości max_depth:

- Wytrenuj model drzewa decyzyjnego
- Oceń dokładność modelu na zbiorze testowym
- Oblicz liczbę fałszywych pozytywów i False Positive Rate (FPR)

2. Porównaj wyniki w formie tabeli
3. Stwórz wykres pokazujący zależność między max_depth a FPR
4. Wybierz optymalną wartość parametru, która zapewnia dobry kompromis między dokładnością a liczbą fałszywych pozytywów

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd

# TODO: Upewnij się, że dane X_train, X_test, y_train, y_test są dostępne
# (mogą pochodzić np. z wcześniejszego zadania)

# Lista wartości parametru max_depth do przetestowania
max_depths = ...  # wypisz gębokości w formacie tablicy [...]

# Lista do zapisu wyników
results = []

# TODO: Dla każdej wartości max_depth:
for depth in max_depths:
    # 1. Wytrenuj model DecisionTreeClassifier z daną głębokością
    clf = DecisionTreeClassifier(max_depth=..., random_state=42)
    clf.fit(...)

    # 2. Wygeneruj predykcje na zbiorze testowym
    y_pred = ...

    # 3. Oblicz macierz konfuzji i False Positive Rate (FPR)
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()

    test_accuracy = ...
    fpr = ...

    # 4. Zapisz wyniki do listy (uzupełnij)
    results.append({
        'max_depth': ...,
        'accuracy': ...,
        'false_positives': ...,
        'false_positive_rate': ...
    })

    # 5. Narysuj drzewo decyzyjne dla wybranych głębokości
    if depth in [3, 5]:
        plt.figure(figsize=(16, 8))
        plot_tree(
            clf,
            feature_names=X_train.columns,
            class_names=['Normalny', 'Złośliwy'],
            filled=True,
            rounded=True
        )
        plt.title(f'Drzewo decyzyjne (max_depth={depth})')
        plt.tight_layout()
        plt.show()

# TODO: Przekonwertuj wyniki do DataFrame i wyświetl jako tabelę
results_df = pd.DataFrame(results)
print("\nWyniki dla różnych wartości parametru max_depth:")
display(results_df)

# TODO: Narysuj wykres słupkowy pokazujący zależność między max_depth a False Positive Rate
plt.figure(figsize=(10, 6))
plt.bar(results_df['max_depth'], results_df['false_positive_rate'], color='red')
plt.title('False Positive Rate w zależności od max_depth')
plt.xlabel('max_depth')
plt.ylabel('False Positive Rate (FPR)')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


## E.1: Enrichment - Pobieranie podstawowych informacji o IP/domenach

W tej części zajmiemy się wzbogacaniem (enrichment) danych o przepływach sieciowych przez dodanie podstawowych informacji o adresach IP, takich jak lokalizacja geograficzna.

### Wzbogacanie danych o informacje geolokalizacyjne
Zadanie:
Wzbogać tabelę z podejrzanymi przepływami o informacje geolokalizacyjne adresów IP. Użyj prostego API do określenia kraju pochodzenia dla każdego adresu IP i dodaj te informacje do tabeli wyników.

In [ ]:
# Import potrzebnych bibliotek
import pandas as pd
import requests

# Prosta funkcja do sprawdzania lokalizacji adresu IP
def get_country(ip_address):
    """
    Pobiera informację o kraju dla adresu IP.

    Args:
        ip_address: Adres IP do sprawdzenia

    Returns:
        str: Nazwa kraju lub 'Nieznany'
    """
    # Pomijamy adresy prywatne
    if ip_address.startswith(('192.168.', '10.', '172.16.', '127.')):
        return 'Adres prywatny'

    try:
        # Używamy darmowego API
        response = requests.get(f'http://ip-api.com/json/{ip_address}', timeout=3)
        data = response.json()

        if data.get('status') == 'success':
            return data.get('country', 'Nieznany')
        else:
            return 'Nieznany'
    except:
        return 'Błąd API'


print("Oryginalna tabela:")
print(stats_traffic)

# Dodanie informacji o kraju dla każdego adresu IP
stats_traffic['src_country'] = suspicious_flows['src_ip'].apply(get_country)
stats_traffic['dst_country'] = suspicious_flows['dst_ip'].apply(get_country)

print("\nTabela wzbogacona o lokalizacje:")
stats_traffic.head()

## V.2: Wizualizacja - Mapa geograficzna lokalizacji podejrzanych adresów IP
W tej części zajmiemy się wizualizacją lokalizacji adresów IP wykrytych jako podejrzane. Dzięki wizualizacji na mapie możemy szybko zidentyfikować, z jakich obszarów geograficznych pochodzi potencjalnie złośliwy ruch.

### Wizualizacja lokalizacji podejrzanych IP na mapie
Zadanie:
Stwórz prostą mapę geograficzną pokazującą lokalizacje adresów IP wykrytych jako podejrzane. Użyj biblioteki folium do stworzenia interaktywnej mapy z markerami wskazującymi lokalizacje.

In [ ]:
# Import potrzebnych bibliotek
import pandas as pd
import folium
import requests

# Funkcja do pobierania lokalizacji adresu IP
def get_ip_location(ip_address):
    """
    Pobiera lokalizację (kraj, szerokość i długość geograficzną) dla adresu IP.

    Args:
        ip_address: Adres IP do sprawdzenia

    Returns:
        dict: Słownik z informacjami o lokalizacji
    """
    # Pomijamy adresy prywatne
    if ip_address.startswith(('192.168.', '10.', '172.16.', '127.')):
        return {'country': 'Adres prywatny', 'lat': 0, 'lon': 0}

    try:
        # Używamy darmowego API
        response = requests.get(f'http://ip-api.com/json/{ip_address}', timeout=3)
        data = response.json()

        if data.get('status') == 'success':
            return {
                'country': data.get('country', 'Nieznany'),
                'lat': data.get('lat', 0),
                'lon': data.get('lon', 0)
            }
        else:
            return {'country': 'Nieznany', 'lat': 0, 'lon': 0}
    except:
        return {'country': 'Błąd API', 'lat': 0, 'lon': 0}

# Przykładowe dane (tylko 25 dla szybkości)
suspicious_flows = stats_traffic.head(25)

# Pobieramy unikalne adresy IP
unique_ips = pd.concat([
    suspicious_flows['src_ip'].drop_duplicates(),
    suspicious_flows['dst_ip'].drop_duplicates()
]).drop_duplicates().tolist()

print(f"Znaleziono {len(unique_ips)} unikalnych adresów IP.")

# Pobieranie lokalizacji dla każdego unikalnego adresu IP
ip_locations = {}
for ip in unique_ips:
    print(f"Pobieranie lokalizacji dla IP: {ip}")
    ip_locations[ip] = get_ip_location(ip)

# Tworzenie mapy
m = folium.Map(location=[0, 0], zoom_start=2)

# Dodanie markerów dla każdego adresu IP
for ip, location in ip_locations.items():
    # Pomijamy adresy bez współrzędnych
    if location['lat'] == 0 and location['lon'] == 0:
        continue

    # Dodanie markera z informacją o adresie IP
    folium.Marker(
        location=[location['lat'], location['lon']],
        popup=f"IP: {ip}<br>Kraj: {location['country']}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

# Zapisanie mapy do pliku HTML
map_file = 'suspicious_ip_map.html'
m.save(map_file)

print(f"\nMapa została zapisana do pliku: {map_file}")
print("Otwórz ten plik w przeglądarce, aby zobaczyć interaktywną mapę.")

# W Google Colab możemy wyświetlić mapę bezpośrednio
try:
    from google.colab import output
    output.serve_file(map_file)
    print("Mapa wyświetlana poniżej:")
    from IPython.display import IFrame
    display(IFrame(src=map_file, width=800, height=600))
except:
    pass  # Nie jesteśmy w Colab

In [ ]:
# Wyświetlenie mapy w Jupyter Notebook
from IPython.display import display, HTML

# Zapisujemy mapę do zmiennej
map_html = m._repr_html_()

# Wyświetlamy mapę bezpośrednio
display(HTML(map_html))